## Unsupervised Capstone Project

### Ten articles with 10 different authors have been selected for this project. The articles are obtained from free Gutenberg Project and they are all Christmas-related.

In [1]:
import timeit
start=timeit.default_timer()

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re

from collections import Counter

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import spacy
nlp = spacy.load('en')
from spacy.lang.en.stop_words import STOP_WORDS

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [6]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

## Load files

In [8]:
filename=['brady.txt','butler.txt','cutting.txt','dawson.txt','dyke.txt',
          'hughes.txt','mitchell.txt','richmond.txt','rinehart.txt','stretton.txt']


In [9]:
def load_text(filename):
    file=open(filename,'r')
    text=file.read()
    file.close
    return text

In [ ]:
def clean_text(text):
    text=text.lower()
    text=text.replace("ï»¿","")
    text=text.replace("\n"," ")
    text=text.replace("*"," ")
    text=text.replace("-"," ")
    text=text.replace("\'s"," is")
    text=text.replace("n\'t"," not")
    text=text.replace("\'m"," am")
    text=text.replace("\'ll"," will")
    text=text.replace("in'","ing")
    text=text.replace("mr.","mr")
    text=text.replace("mrs.","mrs")
    text=text.replace("i'd","i would")
    text=text.replace("p. s.","")
    text=text.replace("n. y.","")
    text=text.replace("dec.","dec")
    text=text.replace("1904.","1904")
    text=text.replace("austin.","austin")
    text=text.replace("co.","co")
    text=text.replace("new york.","new york")
    text=text.replace("kringle.","kringle")
    text=text.replace("dear.","dear ")
    text=text.replace("not.","not ")
    text=text.replace("do.","do ")
    text=text.replace("nan.","nan ")
    text=text.replace("yes.","yes ")
    text=text.replace("song.","song ")
    text=text.replace("boy.","boy ")
    text=text.replace("will.","will ")
    text=text.replace("now.","now ")
    text=text.replace("child.","child ")
    text=text.replace("reconciled.","reconciled ")
    
    text=text.replace('"'," ")
    text=text.replace('_'," ")
    text=text.replace('?'," ")
    text=text.replace('!'," ")
    text=text.replace(','," ")
    text=text.replace(":"," ")
    text=text.replace(";"," ")
    text=text.replace("...","")
       
    text=re.sub("[\(\[].*?[\)\]]"," ",text)
 
    return text

## Clustering

In [ ]:
df = pd.DataFrame()
vectorizer = TfidfVectorizer(stop_words='english',max_features=5)
for i in range(len(filename)):
    text=load_text(filename[i])
    text=clean_text(text)
    text_clean=text.split()
    
    X = vectorizer.fit_transform(text_clean)
    X_np=X.todense()
    X_df=pd.DataFrame(X_np)
    
    author=filename[i].replace('.txt','')
    X_df['author']=author   
    
    df=df.append(X_df)

In [ ]:
df.shape

(72036, 6)

In [ ]:
from sklearn.preprocessing import normalize
x=df.drop(['author'],axis=1)
x=normalize(x)
y=df['author']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, 
                                                    y,
                                                    test_size=0.99,
                                                    random_state=0)

In [ ]:
x_half1, x_half2, y_half1, y_half2 = train_test_split(x, 
                                                      y,
                                                      test_size=0.5,
                                                      random_state=0)

In [ ]:
x1, x2, y1, y2 = train_test_split(x_half1, 
                                  y_half1,
                                  test_size=0.5,
                                  random_state=0)

In [ ]:
x3, x4, y3, y4 = train_test_split(x_half2, 
                                  y_half2,
                                  test_size=0.5,
                                  random_state=0)

## K-Means

In [ ]:
k=10
km = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=1)
km.fit(x_train)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=10, n_init=1, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [ ]:
from sklearn import metrics

In [ ]:
y_pred=km.predict(x_train)
metrics.adjusted_rand_score(y_train, y_pred)

-0.0009772033808455969

In [ ]:
y_pred=km.predict(x_test)   
metrics.adjusted_rand_score(y_test, y_pred)

0.00048634089428723843

In [ ]:
from sklearn.metrics import pairwise_distances

km.fit(x)
labels=km.labels_

score=metrics.silhouette_score(x, labels, metric='euclidean')
print(score)

1.0


In [ ]:
for sample in [x1, x2, x3, x4]:
    model = km.fit(sample)
    labels = model.labels_
    print(metrics.silhouette_score(sample, labels, metric='euclidean'))

1.0
1.0
1.0
1.0


## Mean-Shift

In [ ]:
from sklearn.cluster import MeanShift, estimate_bandwidth
bandwidth = estimate_bandwidth(x_train, quantile=0.2, n_samples=500)
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
ms.fit(x_train)
labels = ms.labels_
cluster_centers = ms.cluster_centers_
n_clusters_ = len(np.unique(labels))
print("Number of estimated clusters: {}".format(n_clusters_))

Number of estimated clusters: 6


In [ ]:
y_pred=ms.predict(x_train)
metrics.adjusted_rand_score(y_train, y_pred)

-0.0009772033808455969

In [ ]:
y_pred=ms.predict(x_test)   
metrics.adjusted_rand_score(y_test, y_pred)

0.00048634089428723843

In [ ]:
for sample in [x1, x2, x3, x4]:
    model = ms.fit(sample)
    labels = model.labels_
    print(metrics.silhouette_score(sample, labels, metric='euclidean'))

1.0
1.0
1.0
1.0


## Spectral Clustering

In [ ]:
from sklearn.cluster import SpectralClustering
sc = SpectralClustering(n_clusters=2)
sc.fit(x_train)

SpectralClustering(affinity='rbf', assign_labels='kmeans', coef0=1, degree=3,
          eigen_solver=None, eigen_tol=0.0, gamma=1.0, kernel_params=None,
          n_clusters=2, n_init=10, n_jobs=None, n_neighbors=10,
          random_state=None)

In [ ]:
y_pred=sc.fit_predict(x_train)
metrics.adjusted_rand_score(y_train, y_pred)

-0.000781183337948944

In [ ]:
def sil (sample):
    model = sc.fit(sample)
    labels = model.labels_
    score=metrics.silhouette_score(sample, labels, metric='euclidean')
    return score

In [ ]:
sil(x1)

## Affinity Propagation

In [ ]:
from sklearn.cluster import AffinityPropagation
from sklearn import metrics
af = AffinityPropagation()
af.fit(x_train)
cluster_centers_indices = af.cluster_centers_indices_
n_clusters_ = len(cluster_centers_indices)
labels = af.labels_
print('Estimated number of clusters: {}'.format(n_clusters_))

In [ ]:
y_pred=af.fit_predict(x_train)
metrics.adjusted_rand_score(y_train, y_pred)

In [ ]:
af.fit(x)
labels=af.labels_

score=metrics.silhouette_score(x, labels, metric='euclidean')
print(score)

In [ ]:
def sil (sample):
    model = af.fit(sample)
    labels = model.labels_
    score=metrics.silhouette_score(sample, labels, metric='euclidean')
    return score

In [ ]:
x1

## Based on the above four (4) clustering techniques, we will select K-Means clustering.

## Unsupervised Feature Generation

In [ ]:
text_list=[]
len_list=[None]*(len(filename)+1)
len_list[0]=0
for i in range(0,len(filename)):
    text=load_text(filename[i])
    text=clean_text(text)
    temp_list=text.split(".")
    len_list[i+1]=len_list[i]+len(temp_list)
    text_list=text_list+temp_list

In [ ]:
len(text_list)

In [ ]:
len_list

In [ ]:
df=pd.DataFrame(index=range(len(text_list)),columns=['author'])
for i in range(0,(len(len_list)-1)):
    author=filename[i].replace('.txt','')
    x=len_list[i]
    y=len_list[i+1]
    df['author'][x:y]=author

In [ ]:
len(text_list)

In [ ]:
df.shape

In [ ]:
len_list

In [ ]:
len(text_list)

## Bag of Words

In [ ]:
vectorizer=CountVectorizer(stop_words="english", 
                           analyzer='word', 
                           ngram_range=(1, 4), 
                           max_df=1.0, 
                           min_df=1, 
                           max_features=6000)

In [ ]:
x = vectorizer.fit_transform(text_list)
x = x.toarray()
x.shape

In [ ]:
y=df['author']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, 
                                                    y,
                                                    test_size=0.25,
                                                    )

## Random Forest

In [ ]:
rfc = ensemble.RandomForestClassifier()
rfc.fit(x_train, y_train)
print('Training set score:', rfc.score(x_train, y_train))
print('\nTest set score:', rfc.score(x_test, y_test))

## Logistic Regression

In [ ]:
lr = LogisticRegression()
lr.fit(x_train, y_train)
print('Training set score:', lr.score(x_train, y_train))
print('\nTest set score:', lr.score(x_test, y_test))

## TFIDF

In [ ]:
vectorizer = TfidfVectorizer(max_df=0.5, # drop words that occur in more than half the paragraphs
                             min_df=2, # only use words that appear at least twice
                             stop_words='english', 
                             lowercase=True, #convert everything to lower case (since Alice in Wonderland has the HABIT of CAPITALIZING WORDS for EMPHASIS)
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             smooth_idf=True #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                            )

In [ ]:
x = vectorizer.fit_transform(text_list)
x = x.toarray()
x.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, 
                                                    y,
                                                    test_size=0.25,
                                                    )

## Random Forest

In [ ]:
rfc = ensemble.RandomForestClassifier()
rfc.fit(x_train, y_train)
print('Training set score:', rfc.score(x_train, y_train))
print('\nTest set score:', rfc.score(x_test, y_test))

## Logistic Regression

In [ ]:
lr = LogisticRegression()
lr.fit(x_train, y_train)
print('Training set score:', lr.score(x_train, y_train))
print('\nTest set score:', lr.score(x_test, y_test))